# 6.10.1 Ecuaciones diferenciales de compatibilidad tridimensionales

|Quién | Fecha | Qué hizo |
| ---  | ---   | ---      |
|Santiago Pérez Londoño <sperezlo@unal.edu.co> | Octubre 4, 2023 | Código inicial                   |
|Diego Andrés Alvarez <daalvarez@unal.edu.co>  | Octubre 5, 2023 | Mejorando la claridad del código |

## Ecuaciones de compatibilidad de Beltrami

Las ecuaciones de Beltrami son las ecuaciones diferenciales de compatibilidad tridimensionales en términos de esfuerzos que asumen que las fuerzas másicas son constantes. Estas resultan al reemplazar en las ecuaciones diferenciales de Saint-Venant la ley de Hooke y asumir que las fuerzas másicas son constantes.

A continuación, se calcularán dichas ecuaciones en coordenadas cilíndricas:

In [1]:
from sympy import sin, cos, diff, Function, simplify, symbols, Matrix, expand

Se definen las variables y funciones de interés:

In [2]:
r, t, z, nu = symbols("r, theta, z, nu")

sr  = Function('sigma_r')(r, t, z)
st  = Function('sigma_θ')(r, t, z)
sz  = Function('sigma_z')(r, t, z)
trt = Function('tau_rθ')(r, t, z)
trz = Function('tau_rz')(r, t, z)
ttz = Function('tau_θz')(r, t, z)

Se define la matriz de transformación $\boldsymbol{T}$:

In [3]:
# Definimos la matriz de transformación
T = Matrix([[ cos(t), -sin(t),     0 ],     
            [ sin(t),  cos(t),     0 ],      
            [      0,       0,     1 ]])

Se define el tensor de esfuerzos en coordenadas cilíndricas $\underline{\underline{\boldsymbol{\sigma}_{\text{cil}}}}$:

In [4]:
sigma_cil = Matrix([[  sr,  trt,  trz ],     
                    [ trt,   st,  ttz ],      
                    [ trz,  ttz,   sz ]])

Se convierte el tensor de esfuerzos $\underline{\underline{\boldsymbol{\sigma}_{\text{cil}}}}$ de la base $\{\hat{\boldsymbol{r}}, \hat{\boldsymbol{\theta}}, \hat{\boldsymbol{z}}\}$ a la base $\{\hat{\boldsymbol{i}}, \hat{\boldsymbol{j}}, \hat{\boldsymbol{k}}\}$ utilizando la ecuación
\begin{equation*}
\underline{\underline{\boldsymbol{\sigma}}} =
\boldsymbol{T} \underline{\underline{\boldsymbol{\sigma}_{\text{cil}}}}
\boldsymbol{T}^T
\end{equation*}

In [5]:
sigma = T*sigma_cil*T.T

Extraemos las componentes de los esfuerzos del tensor $\underline{\underline{\boldsymbol{\sigma}}}$:

In [6]:
sx  = sigma[0,0]
sy  = sigma[1,1]
sz  = sigma[2,2]
txy = sigma[0,1]
txz = sigma[0,2]
tyz = sigma[1,2]

Se define el primer invariante de esfuerzos en coordenadas cilíndricas:

In [7]:
Theta = sr + st + sz

 Definimos las derivadas en coordenadas rectangulares:

In [8]:
dr_dx =    cos(t);   dr_dy = sin(t)
dt_dx = -sin(t)/r;   dt_dy = cos(t)/r

# se definen las primeras derivadas utilizando la regla de la cadena
d_dx = lambda f: diff(f,r)*dr_dx + diff(f,t)*dt_dx
d_dy = lambda f: diff(f,r)*dr_dy + diff(f,t)*dt_dy
d_dz = lambda f: diff(f,z)

# se definen las segundas derivadas
d2_dx2  = lambda f: d_dx(d_dx(f))
d2_dy2  = lambda f: d_dy(d_dy(f))
d2_dz2  = lambda f: d_dz(d_dz(f))
d2_dxdy = lambda f: d_dx(d_dy(f))
d2_dydz = lambda f: d_dy(d_dz(f))
d2_dxdz = lambda f: d_dx(d_dz(f))

Definimos el laplaciano en coordenadas cilindricas:

In [9]:
lapl = lambda f: diff(f,r,2) + diff(f,r)/r + diff(f,t,2)/r**2 + diff(f,z,2)

Escribimos las ecuaciones de Beltrami en coordenadas rectangulares:
\begin{align*}
\left(1+\nu\right) \nabla^2 \sigma_x + \frac{\partial^2\Theta}{\partial x^2} &= 0 & \left(1+\nu\right)\nabla^2 \tau_{yz} + \frac{\partial^2\Theta}{\partial y\partial z} &= 0 \\
\left(1+\nu\right) \nabla^2 \sigma_y + \frac{\partial^2\Theta}{\partial y^2} &= 0 & \left(1+\nu\right)\nabla^2 \tau_{xz} + \frac{\partial^2\Theta}{\partial x\partial z} &= 0 \label{eq:beltrami} \\
\left(1+\nu\right) \nabla^2 \sigma_z + \frac{\partial^2\Theta}{\partial z^2} &= 0 & \left(1+\nu\right)\nabla^2 \tau_{xy} + \frac{\partial^2\Theta}{\partial x\partial y} &= 0
\end{align*}

In [10]:
e1a = (1+nu)*lapl(sx)  + d2_dx2(Theta)  # == 0
e2a = (1+nu)*lapl(sy)  + d2_dy2(Theta)  # == 0
e3a = (1+nu)*lapl(sz)  + d2_dz2(Theta)  # == 0
e4a = (1+nu)*lapl(tyz) + d2_dydz(Theta) # == 0
e5a = (1+nu)*lapl(txz) + d2_dxdz(Theta) # == 0
e6a = (1+nu)*lapl(txy) + d2_dxdy(Theta) # == 0

Se hace $\sin(\theta)=0$ y $\cos(\theta)=1$, como en ocasiones anteriores:

In [11]:
e1a = simplify(e1a.subs({sin(t): 0, cos(t): 1}))
e2a = simplify(e2a.subs({sin(t): 0, cos(t): 1}))
e3a = simplify(e3a.subs({sin(t): 0, cos(t): 1}))
e4a = simplify(e4a.subs({sin(t): 0, cos(t): 1}))
e5a = simplify(e5a.subs({sin(t): 0, cos(t): 1}))
e6a = simplify(e6a.subs({sin(t): 0, cos(t): 1}))

Compararemos las ecuaciones obtenidas con las ecuaciones de Beltrami que aparecen en la literatura:
    \begin{align*}
        (1+\nu)\left(\nabla^2 \sigma_r - \frac{4}{r^2}\frac{\partial \tau_{r\theta}}{\partial \theta} - \frac{2}{r^2}(\sigma_r-\sigma_\theta)\right) + \frac{\partial^2 \Theta}{\partial r^2} &= 0\\
        (1+\nu)\left(\nabla^2 \sigma_\theta + \frac{4}{r^2}\frac{\partial \tau_{r\theta}}{\partial \theta} + \frac{2}{r^2}(\sigma_r-\sigma_\theta)\right) + \frac{1}{r}\frac{\partial \Theta}{\partial r} + \frac{1}{r^2}\frac{\partial^2 \Theta}{\partial \theta^2} &= 0\\
        %
        (1+\nu)\nabla^2 \sigma_z + \frac{\partial^2 \Theta}{\partial z^2} &= 0\\
        %
        (1+\nu)\left(\nabla^2 \tau_{\theta z} + \frac{2}{r^2}\frac{\partial \tau_{rz}}{\partial \theta} - \frac{1}{r^2}\tau_{\theta z}\right) + \frac{1}{r}\frac{\partial^2 \Theta}{\partial \theta \partial z} &= 0\\
        %
        (1+\nu)\left(\nabla^2 \tau_{rz} - \frac{2}{r^2}\frac{\partial  \tau_{\theta z}}{\partial \theta} - \frac{1}{r^2}\tau_{rz}\right) + \frac{\partial^2 \Theta}{\partial r \partial z} &= 0\\ 
        %
        (1+\nu)\left(\nabla^2 \tau_{r\theta} + \frac{2}{r^2}\frac{\partial}{\partial \theta}\left(\sigma_r-\sigma_\theta\right) - \frac{4}{r^2}\tau_{r\theta}\right) + \frac{\partial}{\partial r}\left(\frac{1}{r}\frac{\partial \Theta}{\partial \theta}\right) &= 0
    \end{align*}

In [12]:
e1b = (1 + nu)*(lapl(sr)  - (4/r**2)*diff(trt,t) - (2*(sr-st))/r**2) + diff(Theta,r,2)
e2b = (1 + nu)*(lapl(st)  + (4/r**2)*diff(trt,t) + (2*(sr-st))/r**2) + diff(Theta,r)/r + diff(Theta,t,2)/r**2
e3b = (1 + nu)*lapl(sz)   + diff(Theta,z,2)
e4b = (1 + nu)*(lapl(ttz) + (2*diff(trz,t)/r**2) - ttz/r**2) + diff(diff(Theta,z),t)/r
e5b = (1 + nu)*(lapl(trz) - (2*diff(ttz,t)/r**2) - trz/r**2) + (diff(diff(Theta,z),r))
e6b = (1 + nu)*(lapl(trt) + (2*diff((sr-st),t)/r**2) - (4*trt/r**2)) + diff((diff(Theta,t)/r),r)

Observamos que se obtuvo el vector de ceros, lo que confirma que acabamos de calcular las ecuaciones de Beltrami en coordenadas cilindricas:

In [13]:
expand(Matrix([ e1a - e1b,
                e2a - e2b,
                e3a - e3b,
                e4a - e4b,
                e5a - e5b,
                e6a - e6b ]))

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])